In [6]:
# Class: COEN 140 Lab Machine Learning/Data Mining
# Name: Matthew Davenport
# Date: 9/25/2022
# Title: Lab 8 – K-Means Clustering

import numpy as np
import math

# as directed by lab doc:
num_of_clusters = 10
num_of_initializations = 15

In [7]:
import pickle
from sklearn.datasets import fetch_openml

try: 
    with open("data.pickle", "rb") as fp:
        X_training = pickle.load(fp)
    print("File unpickled successfully")

except FileNotFoundError:
    mnist = fetch_openml('mnist_784', version=1)
    X_training = mnist["data"][:2000]
    
    with open("data.pickle", "wb") as fp:
        pickle.dump(X_training, fp)

print(X_training.shape)

File unpickled successfully
(2000, 784)


my_kmeans performs a k-means clustering of the 2000 images of digits.
Takes 3 arguments

data: the data matrix

K: the number of clusters

M: the number of initializations

Returns: 

(1) the K centroids and cluster assignments for the best solution with the lowest loss 
function (recall that the k-means loss function is the sum of the squared distances of 
observations from their assigned means) 

(2) the sequence of values of the loss-function over k-means iterations for the best solution 
(this should be non-increasing) 

(3) the set of M terminal loss-function values for all initializations   


In [10]:
from matplotlib import pyplot as plt
# def my_kmeans(data, K, M):
#     # return lists: 
#     return_clusters = []      # holds clusters to be returned 
#     return_centroids = []     # centroids to be returned 
#     return_losses = [] 
#     return_final_losses = []  # final losses realized
    
#     data_size = len(data)     # number of points in our dataset
    
#     mean_square_errors = []   # list of mean square errors realized
    
#     data = np.asarray(data)    # convert data to np array
#     # M initializations
#     for i in range(M): 
        
#         initializations = np.random.choice(2000, K, replace=False) # different inits per M iterations
        
#         closest_clusters = np.zeros(data_size)
#         temp_losses = []
#         # initialize centroids list with random points from initializations
#         centroids = np.zeros(data_size)
#         for j in initializations: 
#             centroids[i] = i
        
#         #kmeans algorithm loop: 
#         convergence = True
#         while convergence: 
#             for k in range(data_size):
#                 euclidean_distances =  np.linalg.norm(data[k] - centroids, axis=1) #compute euclidean distance for each point in data to each centroid
#                 closest_cluster = np.argmin(euclidean_distances) # find the min euclidean distance to find which cluster point belongs to
#                 closest_clusters[k] = closest_cluster               
                
#             prev_centroids = np.copy(centroids) # temp copy of the previous array of centroids for comparisons
            
#             loss_val = 0
            
#             for l in range(K):
#                 cluster_points = []
#                 for m in range(data_size):
#                     if closest_cluster[m] == l:
#                         cluster_points.append(data[m])
                        
#                 centroids[l] = np.mean(cluster_points, axis=0)
#                 for point in cluster_points:
#                     loss_val += np.linal.norm(point - centroids[l])
                
#             loss_val /= data_size
#             if np.all(np.equal(prev_centroids, centroids)):
#                 convergence = False
        
#         # update mean square errors with the loss value of the current iteration:        
#         mean_square_errors.append(loss_val)
#         # updating return lists: 
#         return_centroids.append(centroids)
#         return_clusters.append(closest_clusters)
#         losses.append(loss_val)
#         final_losses.append(loss_val[-1])
        
        
#         # Plotting per iteration:
#         print("Plot for iteration: ", (i + 1))
#         plt.plot(loss_val)
#         plt.show()
        
#     # Mean square error plots: 
#     iteration = list(range(0, M - 1))
#     plt.plot(iteration, mean_square_errors)
#     plt.show()
    
#     min = final_losses.index(min(final_losses))
#     return [final_centroids[min], final_clusters[min], losses[min], final_losses]

def my_kmeans(data, K, M):
    """
    Perform a k-means clustering of the 2000 images of digits.

    Args:
        matrix : data matrix containing information about all of our images
        K : number of clusters
        M : number of initializations

    Returns a list containing the following:
        1) The K centroids and cluster assignments for the best solution with the lowest loss function
        2) The (non-increasing) sequence of values of the loss-function over k-means iterations for the best solution
        3) The set of N terminal loss-function values for all initializations
    """
    num_points = len(data)
    
    # need to convert dataset to a 1-D array so we can easily iterate through it
    data = np.asarray(data)
    
    # lists to hold our return values
    final_clusters = []
    final_centroids = []
    losses = []
    final_losses = []
    
    # list to hold our MSEs for the 15 iterations
    MSEs = []
    
    # run outer for loop depending on M initializations
    for i in range(M):
        # defines which of the K centroids each row of the matrix is closest to
        class_list = np.zeros(num_points)
        
        # initialize our random K centroids
        inits = np.random.choice(2000, K, replace=False)
        
        # fill initial centroid list with K random points from the matrix
        centroids = np.asarray([data[index] for index in inits])

        loss = []
        while True:
            # calculate the distance of all instances to the K centroids and assign instances to closest centroid
            # in other words, recompute cluster membership
            for j in range(num_points):
                # caluclate the Euclidean distance for all of our data points
                distances = np.linalg.norm(data[j]-centroids, axis=1)

                # assign each data point to the closest cluster centroid
                cluster = np.argmin(distances)
                
                # keep track of which cluster each data point is in
                class_list[j] = cluster
                
            # make a copy of our current centroids so we can compare to the new ones later
            test = np.copy(centroids)
            
            #initialize our loss value for this iteration
            loss_value = 0
            
            # calculate our loss function; recompute the K centroids
            for j in range(K):
                # find all points associated with respective centroid
                cluster_points = []
                for m in range(num_points):
                    if class_list[m] == j:
                        cluster_points.append(dataset[m])
                
                # calculate our new centroids
                centroids[j] = np.mean(cluster_points, axis=0)
                
                # calculate mean square error (MSE) for each point
                for item in cluster_points:
                    loss_value += np.linalg.norm(item-centroids[j])
            loss_value = loss_value / num_points
            loss.append(loss_value)
            
            # if all new centroids are equal to the old centroids, we can stop our while loop; this indicates convergence
            if np.all(np.equal(test, centroids)):
                break
        
        # MSE will be final loss value of respective iteration
        MSEs.append(loss_value)
                
        # append and plot our results
        final_centroids.append(centroids)
        final_clusters.append(cluster_points)
        losses.append(loss)
        final_losses.append(loss[-1])
        
        # plot our loss results for the respective iteration
        plt.plot(loss)
        plt.title("Iteration: {}".format(i+1))
        plt.xlabel("K-value")
        plt.ylabel("Loss")
        plt.show()
        

    # plot our MSEs over the M iterations
    x = []
    for iteration in range(M):
        x.append(iteration)
    plt.plot(x, MSEs)    
    plt.title("MSE Values over M Iterations")
    plt.xlabel("Iteration")
    plt.ylabel("MSE")
    plt.show()
    
    min_index = final_losses.index(min(final_losses))
    best_values = [final_centroids[min_index], final_clusters[min_index], losses[min_index], final_losses]
    return best_values

In [11]:
kmeans_results = []
kmeans_results = my_kmeans(X_training, num_of_initializations, num_of_clusters)


ValueError: operands could not be broadcast together with shapes (784,) (2000,) 